# Mesh

See [MIKE IO Documentation](https://dhi.github.io/mikeio/dfsu.html)

In [ ]:
import matplotlib.pyplot as plt
from mikeio import Mesh

## A simple mesh

Let's consider a simple mesh consisting of 2 triangular elements. 

In [ ]:
fn = "data/two_elements.mesh"

In [ ]:
with open(fn, "r") as f:
    print(f.read())

In [ ]:
msh = Mesh(fn)
msh

In [ ]:
msh.plot();

In [ ]:
msh.node_coordinates

In [ ]:
msh.element_table

In [ ]:
msh.element_coordinates

In [ ]:
msh.get_element_area()

Let's plot the node and element coordinates:

In [ ]:
xn, yn = msh.node_coordinates[:,0], msh.node_coordinates[:,1]
xe, ye = msh.element_coordinates[:,0], msh.element_coordinates[:,1]

ax = msh.plot()
ax.plot(xn, yn, 'ro', markersize=10)
ax.plot(xe, ye, 'bx', markersize=10)

### Boundary polylines

It can sometimes be convenient to have mesh boundary as a polyline (or multiple in case of more complex meshes). 

In [ ]:
msh.boundary_polylines

In [ ]:
xy = msh.boundary_polylines.exteriors[0].xy
plt.plot(xy[:,0], xy[:,1])
plt.axis("equal");